##### **Get Scryfall cards**

In [ ]:
from art_scan import get_scryfall_cards

queries = [
    "City of Brass",
]

for q in queries:
    cards = get_scryfall_cards(q)
    [print(f"\t{repr(c)}") for c in cards]

##### **Check MTGPICS for images**

In [ ]:
from art_scan import get_scryfall_cards, get_mtgpics_art_ids, save_mtgpics_image_alt

queries = [
    "City of Brass",
]

for q in queries:
    cards = get_scryfall_cards(q)
    print(f"Found: {get_mtgpics_art_ids(cards)}")

##### **Download card images from Scryfall queries**

In [ ]:
from art_scan import process_query

queries = [
    "Crucible of Worlds"
]

[process_query(q) for q in queries]
# [process_query(q, force_scryfall=True, skip_mtgpics=True) for q in queries]

##### **Generate inventory to CSV**

In [ ]:
from dataclasses import dataclass
import os
from lib import os_helpers


@dataclass
class InventoryCard:
    name: str
    modified: str
    type_: str
    
    def __str__(self):
        return f"{self.name},{self.type_},{self.modified}"


proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"
inventory_file_name = "inventory.csv"

# Get all cards
inventory_cards = []
for card_type in os.listdir(proxy_folder):
    # Skips any files (folders won't have periods)
    if "." in card_type:
        print(f"Skipping file ({card_type})..")
        continue

    typed_proxy_folder = f"{proxy_folder}/{card_type}"
    for file_name in os.listdir(typed_proxy_folder):
        card = InventoryCard(
            name=".".join(file_name.split(".")[:-1]),  # Strip off file extension
            modified=os_helpers.get_modified_date(f"{typed_proxy_folder}/{file_name}"),
            type_=card_type
        )
        inventory_cards.append(card)

# Write list of cards to CSV
with open(f"{proxy_folder}/{inventory_file_name}", "w") as f:
    for card in inventory_cards:
        f.write(str(card) + "\n")


##### **Pull Private Proxy List from Google Sheets**

In [ ]:
from googleapiclient import discovery
from lib.classes import Proxy
from lib.google_auth import get_creds

args = dict(
    spreadsheetId='13ofTLZFmPMkT09Np2US8KF_JmTfgqdeQ5aeY16L43s0',
    range='Inventory!A:L',
)

service = discovery.build('sheets', 'v4', credentials=get_creds())
result = service.spreadsheets().values().get(**args).execute()
proxy_csv_data = list(result.get('values', []))

all_proxies = [Proxy.from_row(p) for p in proxy_csv_data[4:] if len(p[0]) > 0]
proxies_to_print = [p for p in all_proxies if p.order_count > 0]
len(proxies_to_print)

##### **Pull Public Proxy List from Google Sheets**

In [ ]:
from lib.classes import Proxy
from lib import common

sheet_id = '1JbJy3kM34XUOPud4qPaAhp2luU0Qgidq4CPQVm0TTw0'
url = f'https://docs.google.com/spreadsheet/ccc?key={sheet_id}&output=csv'

proxy_csv_data = common.load_csv(url.format(sheet_id))

all_proxies = [Proxy.from_row(p) for p in proxy_csv_data[4:] if len(p[0]) > 0]
proxies_to_print = [p for p in all_proxies if p.order_count > 0]
len(proxies_to_print)

##### **Find missing proxy files**

In [ ]:
from lib import os_helpers

proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

card_types = list(set([p.type for p in proxies_to_print]))
proxies_by_type = {
    type_: [p for p in proxies_to_print if p.type == type_]
    for type_ in card_types
}

missing_files = []
for card_type in card_types:
    folder_name = f"{proxy_folder}/{card_type}"
    proxy_file_names = [p.name for p in proxies_by_type[card_type]]
    missing_files += os_helpers.find_missing_files(folder_name, proxy_file_names)

print("\n".join(missing_files))

##### **Write out files for order**

In [ ]:
proxies = [p for p in proxies_to_print if not p.is_mdfc]
# proxies = [p for p in proxies_to_print if p.is_mdfc]
len(proxies)

In [ ]:
from lib import os_helpers, proxy_helpers
import importlib

importlib.reload(os_helpers)
importlib.reload(proxy_helpers)

folder_size = 612
prefix_name = "proxies_{number:02d}"
landing_folder = "C:/Users/evanh/Temp/landing"
output_folder = "C:/Users/evanh/Temp/mpc"
proxy_folder = "C:/Users/evanh/My Drive/MTG/Proxies/mpc-ready"

# Write all files out to landing zone
missing, duplicates = proxy_helpers.create_unique_proxies(proxies, proxy_folder, landing_folder)
if len(missing):
    print("Missing:\n\t{}".format('\n\t'.join([p.name for p in missing])))
if len(duplicates):
    print("Duplicates:\n\t{}".format('\n\t'.join([p.name for p in duplicates])))

# Move files into subfolders of fixed size (`folder_size`)
output_folders = os_helpers.split_files(landing_folder, output_folder, folder_size, prefix_name)
output_folders